# Crop Classification

The project focus three stages of crop cylce, please follow below link for know about the stages. 

[Crop details](https://docs.google.com/drawings/d/1oyH4NqZqckdJBXbudCC-5BtxOSA5LPmg3pojAGm8RP4/edit?usp=sharing)

Project focuses on 7 classes of crops which is divided into 4 different categories depending upon the respective crop cycle.


| crop      | crop_class |
| ----------- | ----------- |
| barley_w      | 1       |
| wheat_w   | 2        |
| rapeseed_w      | 3       |
| rye_w   | 4        |
| barley_s      | 5       |
| wheat_s   | 6        |
| oats_s   | 7       |



## Know the crop data

#### Import

In [1]:
import pandas as pd
import numpy as np
import rioxarray
import os
import geopandas as gpd
import rasterio
import fiona

import rasterio.mask
import sys
from pathlib import Path

from tqdm import tqdm

from data_op import shapefile
from config import settings as st

In [5]:
crop_classes = pd.read_excel(os.path.join(st.data_directory,'crop_code/croptable_2020.xlsx'))

In [6]:
crop_classes

,Crop_nr,Name
0,1,Spring barley
1,2,Spring wheat
2,3,Spring oats
3,4,Mix of spring sown cereals
4,5,Maize for maturity
...,...,...
321,711,Green grain of winter triticale
322,900,Other crops
323,903,Light and open areas in wood
324,905,Other use on areas with schemes


In [9]:
crop_classes_list = np.unique(crop_classes.Crop_nr)
print(len(crop_classes_list))

326


Distinct crop classes and names for 7 crops, divided into 4 categories depending upon the respective crop cycle, 

**Categories**

| Index      | Months |
| ----------- | ----------- |
| 1      |  Jan-Mar      |
| 2   | Apr-Jun        |
| 3      | Jul-Sep       |
| 4   | Oct-Nov        |

**Category wise crop classes**

</br>
</br>

**1. Jan-Mar**

| crop      | Crop_nr |   cycle_stage     |
| ----------- | ----------- | ----------- |
|  barley_w     |   706     | mid |
| wheat_w   | 707       | mid |
| rye_w      | 709,710      | mid |

</br>
</br>

**2. Apr-Jun**

| crop      | Crop_nr |   cycle_stage      |
| ----------- | ----------- | ----------- |
|  barley_w     |   706     | mid |
| wheat_w   | 707       | mid |
| rye_w      | 709,710      | mid |
| barley_s   | 01,701      | planting and mid |
| wheat_s      | 02,06,702     | planting and mid  |
| oats_s      | 03,703    | planting and mid  |

</br>
</br>


**3. Jul-Sep**

| crop      | Crop_nr |   cycle_stage      |
| ----------- | ----------- | ----------- |
|  barley_w     |   706,220     | mid and harvest |
| wheat_w   | 707,221       | mid and harvest|
| rye_w      | 222     | harvest |
| barley_s   | 701,210      | mid and harvest |
| wheat_s      | 702,211     | mid and harvest |
| oats_s      | 703,212    | mid and harvest  |

</br>
</br>


**4. Oct-Dec**

| crop      | Crop_nr |   cycle_stage      |
| ----------- | ----------- | ----------- |
|  barley_w     | 10    | planting|
| wheat_w   | 11,13      | planting|
| rapeseed_w      | 22     | planting |
| rye_w      | 14,15     | planting |
| barley_s   | 210      | harvest |
| wheat_s      | 211     | harvest |

</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>

Summmer and Winter crop categorisation

| Season      | crop |
| ----------- | ----------- |
| Winter      | barley_w, wheat_w, rapeseed_w, rye_w       |
| Summer/Spring   | barley_s,wheat_s,oats_s        |


## Prepare Dataset



In [2]:
# Create 4 shapefiles from 4 categorise 
path_raw = 'Raw_shape_files/Raw_downloaded_files'
shape_ex = shapefile.Shape_Extractor(shape_file = os.path.join(st.data_directory,path_raw,'Denmark_shape_2020/Marker_2020.shp'))
denmark_veg = shape_ex.import_shape()


In [3]:
columns_to_be_remove= ['Marknr', 'IMK_areal', 'Journalnr', 'CVR', 'Afgroede', 'GB','Markblok', 'GBanmeldt']
denmark_veg = shape_ex.filter_shape_data(df = denmark_veg,column_names = columns_to_be_remove)

In [23]:
#rename column 
denmark_veg.reset_index(drop=True, inplace=True)

In [4]:
classes_for_cat_1 = {
    'barley_w': [706],
    'wheat_w': [707],
    'rye_w': [709,710]
    }
denmark_veg_cat_1 = shape_ex.prepare_labels(df = denmark_veg,class_dictionary = classes_for_cat_1,id_col = 'code',label_name='classes')

Index(['Marknr', 'IMK_areal', 'Journalnr', 'CVR', 'Afgkode', 'Afgroede', 'GB',
       'Markblok', 'GBanmeldt', 'geometry'],
      dtype='object')

In [24]:
denmark_veg.columns

Index(['Afgkode', 'geometry'], dtype='object')

In [26]:
denmark_veg.rename(columns = {'Afgkode':'code'},errors='raise')

,code,geometry
0,318.0,"POLYGON ((447095.808 6232154.360, 447097.965 6..."
1,318.0,"POLYGON ((447414.977 6232743.568, 447416.902 6..."
2,254.0,"POLYGON ((447968.632 6232132.837, 447968.701 6..."
3,250.0,"POLYGON ((463635.528 6233882.395, 463592.872 6..."
4,254.0,"POLYGON ((445791.067 6243928.038, 445791.721 6..."
...,...,...
583669,NaN,"POLYGON ((511539.465 6161423.150, 511531.294 6..."
583670,NaN,"POLYGON ((511622.497 6161362.253, 511618.345 6..."
583671,NaN,"POLYGON ((479566.600 6172007.170, 479566.690 6..."
583672,NaN,"POLYGON ((511350.042 6160285.812, 511350.561 6..."
